**Project: Visualizing TTL Behavior and Latency Spikes Using Traceroute**

**1. Introduction**

This project explores how the Time-To-Live (TTL) value in IP packets affects routing behavior and how tools like Tracert can be used to identify:

The number of network hops between a source and destination

Network segmentation or firewall inspection points

Latency spikes that could indicate geographic distance, congestion, or inspection

Tracert works by sending packets with increasing TTL values, allowing us to discover each hop along the path to a target. We will use this behavior to analyze a real-world path and draw conclusions about the route's structure.

**2. Project Goal**

Map the network path (hops) from this system to a public host

Identify latency spikes, which may indicate:

Firewalls

Geographical distance

Network congestion

Reinforce how TTL expiration and ICMP responses reveal hop-by-hop routing

**3. Tools Used**

Using Nmap which includes built-in traceroute functionality as part of its scanning process 

Public destination: google.com (or another reachable, reliable domain)

**4. Step-by-Step Procedure**

**Step 1: Run Tracert Command**
Windows:

tracert google.com



**Step 2: Observe the Output**
For each hop:
Record the hop number
Record the IP address or hostname
Record the three latency values (in ms)
Note any hops that return * * * (timeouts)

**Step 3: Identify Patterns**

Look for:
Sudden increases in latency (e.g., from 20ms to 150ms)
This may suggest a long-distance link or a firewall inspection point
Timeouts or missing hops
This can indicate ICMP suppression (common in hardened networks)
Hostnames that indicate geographic routing (e.g., .sjc, .fra, .sea)

**5. Analysis Questions**

How many hops does it take to reach the destination?

Where do latency spikes occur? What might explain them?

Are any hops hidden or non-responsive?

Do any hostnames suggest the geographic route?

Based on TTL behavior and latency, can we infer any firewalls, segmentation, or traffic shaping?




**Traceroute to google.com via tracert (ICMP Echo Request) — Hop-by-Hop Output and Analysis**

**Hop 1**
Output:
IP: No response
Hostname: Unavailable
Latency: * * *
Result: Request timed out

Explanation: This hop is almost certainly the default gateway (your home router), but it did not respond to the tracert command. On Windows, tracert uses ICMP Echo Request packets with gradually increasing TTL values. When TTL reaches 0, routers are expected to reply with an ICMP "Time Exceeded" message. However, many routers — particularly home or ISP-managed routers — are configured to drop TTL-expired ICMP packets silently, either for security or to reduce processing load. This causes the hop to appear as a timeout (* * *).

In contrast, Nmap's traceroute (as seen in the aggressive scan) uses TCP SYN packets (in this case, to port 3389) with increasing TTL values. When TTL reaches 0, routers are still expected to send back an ICMP "Time Exceeded" message. Many devices that block ICMP Echo Requests still allow ICMP Time Exceeded replies for TCP traffic, which is why the same router responded in the Nmap traceroute but not in tracert. This illustrates how different traceroute methods reveal different levels of visibility depending on protocol filtering behavior.

**Hop 2**
Output:
IP: 2001:506:6000:11b:69:235:122:100
The IP address at Hop 2 is 2001:506:6000:11b:69:235:122:100, which is an IPv6 Global Unicast Address. The prefix 2001:506::/32 is registered to U.S. Federal Government infrastructure via ARIN, although it may be routed through a commercial ISP. The lack of a reverse DNS record (hostname unavailable) is typical for backbone routers or infrastructure segments that do not expose hostnames publicly.

IPv6 addresses are made up of 8 groups of 4 hexadecimal digits (128 bits total), separated by 7 colons. However, IPv6 allows for shorthand to make addresses more readable. Leading zeros can be dropped (e.g., 0069 becomes 69), and long strings of consecutive 0000 groups can be replaced with a double colon (::). This :: can only appear once per address, and its purpose is to reduce clutter when large parts of the address contain zeros. Therefore, different-looking IPv6 addresses (like those at Hop 2 and Hop 4) may be equally valid — one is fully expanded, the other is compressed.


Hostname: Unavailable  
**Note on Hostname Unavailable:**
Hostnames are resolved via reverse DNS (rDNS) lookups, where the system attempts to find a domain name associated with an IP address. If the network device (or its administrator) has not configured reverse DNS records for that IP, or if the responding device blocks DNS resolution, then the hostname will appear as unavailable. This is common on backbone routers, firewalls, or hardened systems where hostname exposure is intentionally suppressed for privacy or security reasons.

A backbone router is a high-performance router that forms part of the core internet infrastructure, connecting ISPs and data centers across regions. These routers carry extremely high volumes of traffic and are often not configured with reverse DNS records, which is why their hostnames frequently appear as unavailable in traceroute output.

Latency: 95 ms, 23 ms, 23 ms
Explanation: This hop responded successfully. The first probe (95 ms) shows noticeably higher latency than the second and third probes (~23 ms), which suggests:

A delayed ICMP reply on the first packet

Or transient network congestion during that moment. This router is likely a regional ISP node — reachable and not filtered. The consistency of the second and third probes implies a stable connection once the initial delay passed.

**Hop 3**
Output:
IP: 2001:506:6000:11b:69:235:122:97
Hostname: Unavailable
Latency: 97 ms, 24 ms, 24 ms
Explanation: Same pattern as Hop 2 — the first response is delayed (97 ms), but the following two are consistent. These hops may belong to the same ISP backbone segment, and the latency variation could be due to:

ICMP response handling priority (lower than data traffic)

Load balancing or a transient queue delay. This hop is responsive and reachable, with no evidence of filtering.

**Hop 4**
Output:
IP: 2001:1890:f6:1691::2
Hostname: Unavailable
Latency: 98 ms, 25 ms, 27 ms

Explanation:
Similar behavior as previous hops — one delayed response and two stable. This hop likely represents a larger transit provider. The ~25–27 ms RTT is consistent with a regional or Tier 1 backbone router.

**Hop 5**
Output:
IP: 2001:1890:fff:fa8:12:255:10:240
Hostname: Unavailable
Latency: 99 ms, 26 ms, 29 ms

Explanation: All three probes at Hop 5 received replies, with one high-latency result (99 ms) followed by two lower, stable responses (26 ms and 29 ms). This brief variation is called jitter — a small, momentary fluctuation in packet delay. Since all responses were returned, and there were no signs of packet drops or timeouts, we can reasonably conclude that there is no blocking or ICMP rate-limiting occurring at this hop.

The IP address falls within a block (2001:1890::/32) registered to AT&T, a major internet provider. Based on its position in the path (Hop 5 of 11), and its responsiveness, this device likely serves as a midpoint router, facilitating traffic between a regional ISP backbone and a major cloud provider such as Google. The absence of a hostname is common for backbone or transit routers that do not publicly advertise reverse DNS records.



**Hop 6**
Output:
IP: 2607:f8b0:8319::1
Hostname: Unavailable
Latency: 98 ms, 26 ms, 26 ms
Explanation: Consistent, low-latency responses — this is likely an edge router for Google, just before the final set of internal hops. No sign of filtering or delay.

**Hop 7
Output:
IP: 2001:4860:0:1::5950
Hostname: Unavailable
Latency: *, 100 ms, *

Explanation: Only 1 out of 3 probes received a reply. This hop is likely configured with ICMP rate-limiting or selective packet filtering. It allows some TTL-expired packets through, but drops others to reduce load or hide details. This behavior is common at hardened network boundaries, especially around high-value infrastructure (e.g., Google's internal systems or peering routers).

**Hop 8**
Output:
IP: 2001:4860:0:1::834a
Hostname: Unavailable
Latency: 99 ms, 29 ms, 26 ms

Explanation: Stable and normal responses. This hop is within Google’s infrastructure, showing low jitter and no signs of delay or filtering.

**Hop 9**
Output:
IP: 2001:4860:0:1::7d4d
Hostname: Unavailable
Latency: 100 ms, 28 ms, 29 ms

Explanation: Continuation through Google’s internal routing fabric, with consistent latency. All probes returned successfully.

**Hop 10**
Output:
IP: 2001:4860:0:1::5fbd
Hostname: Unavailable
Latency: 100 ms, 30 ms, 26 ms

Explanation: Similar performance as Hops 8 and 9. Stable, low-latency, and fully responsive, indicating well-optimized internal routing.

**Hop 11 (Final Destination)**
Output:
IP: 2607:f8b0:4005:803::200e
Hostname: sfo03s33-in-x0e.1e100.net
Latency: 27 ms, 26 ms, 26 ms

Explanation: The final destination (Google.com) was successfully reached. The hostname includes sfo, which likely stands for San Francisco, suggesting the closest Google edge server was selected. All replies were consistent and fast, indicating high availability and low-latency access.

The final destination (google.com) was successfully reached at Hop 11. The hostname sfo03s33-in-x0e.1e100.net indicates a Google-operated edge server located in the San Francisco region, selected via Anycast routing to provide low-latency access. The domain 1e100.net is Google’s internal infrastructure domain, confirming authenticity. All three latency responses were consistent (26–27 ms), demonstrating excellent availability and connection stability.



**Project Conclusion: Visualizing TTL and Latency with Traceroute**

**Goals:** 

The goal of this project was to use the Windows tracert command to:

Visualize the network path (hops) from a local system to google.com

Observe how TTL (Time To Live) expiration reveals intermediate routers

Identify latency spikes, timeouts, and potential firewalls or filtering

Understand how network segmentation or hardened infrastructure impacts visibility

**What Was Discovered:**

Hop 1 (Local Router) did not respond to ICMP-based traceroute, confirming that TTL-expired ICMP Echo Requests are filtered — a common default in consumer and ISP routers. However, a follow-up Nmap TCP-based traceroute confirmed that the router was functional and simply applying protocol-specific filtering, an important real-world security behavior.

Hops 2–6 responded reliably, though occasional jitter (e.g., Hop 5) suggested normal queuing behavior rather than performance issues. These hops reflect the ISP backbone infrastructure and regional peering points.

Hop 7 returned only one of three probes, suggesting ICMP rate-limiting or packet filtering, likely marking a firewall inspection point or hardened router.

Hops 8–10 were stable and responsive, consistent with Google’s internal edge network — indicating well-optimized cloud infrastructure.

Hop 11 was the final destination, a Google edge server in San Francisco, confirming Anycast routing and a clean path with low latency and no packet loss.

Key Takeaways:
TTL-based tools like tracert can expose a great deal about network structure, latency patterns, and filtering behaviors — even when IP hostnames are unavailable.

Different tools (ICMP vs. TCP traceroute) reveal different visibility levels, which is critical for security analysis and infrastructure testing.

IPv6 routing is widely adopted (as seen in this project), and its format, compression, and allocation are worth understanding for modern network analysis.

Concepts like jitter, rate-limiting, and backbone routing emerged naturally from the data, providing rich insight into how real-world internet routing behaves.

